# Outline

In this file we construct the convex polytopal region for a stability condition $\sigma_{\Gamma}$ (in most cases this will be the one given by having $\sigma_{\Gamma}(\Gamma_i)=\underline{0}$ for all spanning trees $\Gamma_i$. Denote this polytope by $R_\sigma^{\Gamma}$ for line bundle multidegrees and $R_\sigma^{\Gamma_i}$ for assignments on trees $\Gamma_i$.

We focus on the cases: $G62,G63,G72,G73, K_{33}$ and $GSym_3$

We aim to answer the following questions:

- For cases get Get random phi that works for ibd case (yes) Done.
- Obtain by hand for G62 and G63 get $R_\sigma^{\Gamma_i}$ (pdf). For G62 and G63:
    - Is $R_\sigma^{\Gamma}= R_\sigma^{\Gamma_i}$ and do they contain each other.
    - What are dim $R_\sigma^{\Gamma_i}$ are any polytopes are empty.
    






## Secondary task

Using spannningtrees_chipadd.ipynb

- We take breaks divisors to construct the $R_\sigma$ for line bundles. 
- We want to do this as dim$R_\sigma$ will be suggestive of dim$R_\sigma^{\Gamma_i}$ if nicolas conjecture is true. 
- From here we either have a function to get all $R_\sigma^{\Gamma_i}$ or do for selected trees by hand


# Functions:

In [23]:
import numpy as np
import itertools
import pickle

In [24]:
def get_formatted_hyperplanes_txt(lbm,phi_G): #put into format of sage b<Ax<b+1
    
    """
    Inputs:
    phi  records indexs of cuts with cutfraction in inequalities.        phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}
    lbm= line bundle mulidegrees given by chip-adding
    
    Purpose: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
    
    
    """
    n=len(lbm[0])
    # We add the total degree equation to inequals
    deg=sum(lbm[0])
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    inequals=[tot_equ1,tot_equ2]     
    
    for d in lbm: #creates row at a time
        for indexor in list(phi_G.keys()):
            
            def sum_comps(comp,indexor): #Done
                """returns the sum of the components of comp in the position by indexor"""
                #for d and phi
                sum_l=[comp[int(i)-1] for i in indexor]
                return sum(sum_l)
            
            
            cf=phi_G[indexor] # want value
            constant_indx= sum_comps(d,indexor)        
        
            l_i= [ [int(x)-1,] for x in indexor] #x,y,z.. terms for indexors in Rf space #Indexs for phi keys
            

            def lower_part(constant_indx,l_i,indexor,cf,n):
                
                bterm=[constant_indx+cf] # as list
                base=np.zeros(n)
                l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            def upper_part(constant_indx,l_i,indexor,cf,n):
                
                #Remember upper part of modulus inequality.

                bterm=[-constant_indx+cf] # as list
                base=np.zeros(n)
                l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            res_lower=lower_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_lower",res_lower)
            res_upper=upper_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_upper",res_upper)
            res_combined= [res_lower,res_upper]
            
            # print("--------")

            inequals=inequals+res_combined
        
    return inequals

In [25]:
#Modified to include total degree equation and work for trees wrt phi inequalites for G62 G63 
# not the same as phi inequal dicts for lbm.

def mod_get_formatted_hyperplanes_txt(lbm,phi_G): #put into format of sage b<Ax<b+1
    
    "Following doc phi-polytope inequalities"
    
    """
    Inputs:
    phi  records indexs of cuts as keys, and adjuster in modulus as value, Cf =0.5 in all cases. 
    lbm= line bundle mulidegrees given by chip-adding
    
    Purpose: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
     
    to add the total degree inequality x+y+z..=degree.
    add in ()
    
    """
    n=len(lbm[0])
    # We add the total degree equation to inequals
    deg=sum(lbm[0])
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    inequals=[tot_equ1,tot_equ2]     
    for d in lbm: #creates row at a time
        for indexor in list(phi_G.keys()):
            
            def sum_comps(comp,indexor): #Done
                """returns the sum of the components of comp in the position by indexor"""
                #for d and phi
                sum_l=[comp[int(i)-1] for i in indexor]
                return sum(sum_l)
            
            cf=0.5 #for phi tree polytopes.
#             cf=phi_G[indexor] # want value
#             constant_indx= sum_comps(d,indexor)   
            constant_indx=phi_G[indexor] #for phi tree polytopes (remeber the minus) #!The adjustor.
        
            l_i= [ [int(x)-1,] for x in indexor] #x,y,z.. terms for indexors in Rf space #Indexs for phi keys
            

            def lower_part(constant_indx,l_i,indexor,cf,n):
                
                bterm=[cf-constant_indx] # as list
                base=np.zeros(n)
                l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat                
                return ineq

            def upper_part(constant_indx,l_i,indexor,cf,n):
                
                #Remember upper part of modulus inequality.

                bterm=[cf+constant_indx] # as list
                base=np.zeros(n)
                l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor.
                
                for item in l:
                    ipos = int(item[1])-1 # ith position
                    val=item[0]
                    base[ipos]=val
                
                xyz=base
                
                ineq=bterm+list(xyz) # [b,x,y,z...] want to concat
                
                return ineq

            res_lower=lower_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_lower",res_lower)
            res_upper=upper_part(constant_indx,l_i,indexor,cf,n)
            
            # print("res_upper",res_upper)
            res_combined= [res_lower,res_upper]
            
            # print("--------")

            inequals=inequals+res_combined
    return inequals

In [26]:
def poly_size(data):
    Poly=Polyhedron(ieqs=data)
    dimP=dim(Poly)
#     print(f"The dimension of the polytope is: {dimP}")
    return dimP

In [27]:
def polytope_size_checker(data,phi,size):
    
    """
    Inputs:
    data: is a list of line bundle multidegrees for the weak stabiltiies that fail when phi is taken to be the average.
    phi= is the data of the cuts and the indexes taken for the phi inequalities.
    size: the size of the nondegenerate polytope (number of vertices)
    """
    recorder_minus=[]
    recorder_0=[]
    recorder_1belowtop=[]
    recorder_max=[]
    dimensions=[]

    counter=0
    for lbm in data:
        lbm_data=get_formatted_hyperplanes_txt(lbm,phi)
        dimp=poly_size(lbm_data)
        if dimp ==0:
            recorder.append(lbm)
        if dimp >0 and dimp<size:
            recorder_1belowtop.append(lbm)
        if dimp ==size:
            recorder_max.append(lbm)
        if dimp==-1:
            recorder_minus.append(lbm)
            
        print(counter)
        dimensions.append(dimp)
        counter+=1

    print("Total number of cases:",len(data))
    print("recorder_minus",len(recorder_minus))
    print("recorder_0:",len(recorder_0))
    print("recorder_1belowtop:",len(recorder_1belowtop))
    print("recorder_max:",len(recorder_max))
    
    print("\n", f"Number of cases of dim: -1:{dimensions.count(-1)}, 0: {dimensions.count(0)}, 1:{dimensions.count(1)}, 2:{dimensions.count(2)}, 3:{dimensions.count(3)}, 4:{dimensions.count(4)}, 5:{dimensions.count(5)}, 6:{dimensions.count(6)}, 7:{dimensions.count(7)}")
          
    return recorder_minus,recorder_0,recorder_1belowtop,recorder_max

In [28]:
def mod_where_specific_lbm_fail(dict,lbm,phi):    
    """
    Checks if a given phi (not average phi) fails.
    
    Outputs where it fails or outputs f"The phi: {} does not fail".
    """
    
    def sum_comps(comp,indexor): #Done
        """returns the sum of the components of comp in the position by indexor"""
        #for d and phi

        sum_l=[comp[int(i)-1] for i in indexor]
        return sum(sum_l)

    def cutfrac(dict,indexor): #Done
        cutfrac=dict[indexor]
        return cutfrac
    
    counter=0
    for d in lbm:
        for indexor in dict.keys():
            if abs(sum_comps(d,indexor) - sum_comps(phi,indexor))<cutfrac(dict,indexor):
                pass
            else:
                print(f"Multidegree: {d}, \n Phi: {phi}, \n Indices summed {indexor}, \n Failed the inequality |{sum_comps(d,indexor)} - {sum_comps(phi,indexor)}|<{cutfrac(dict,indexor)} \n")
                counter+=1
                
    if counter !=0:
        print(f"The phi (center of polytope): {phi} fails the phi-inequalities")
    if counter ==0:
        print(f"The phi (center of polytope): {phi} does not fail the phi-inequalities")
        
    def get_phi_av(lbm): #Done
        lbm=[np.array(d) for d in lbm]
        length=len(lbm)
        slbm=sum(lbm)
        phi=slbm/length
        return phi

    print(f"phi average is: {get_phi_av(lbm)}")

    return 

In [29]:
def get_a_phi(poly):
#     phi=poly.center()
    phi=poly.representative_point()
    return phi

# Polytopes by phi inequalites for trees (0 assignment)

## G62 and Comparision with polytope by phi lbm inequalities (IBD)

In [30]:
#Load tree phi inequalities

G62_tree_phi_inequal=[{"2":1,"23":1,"234":1,"1":0.5,"2345":1},
{"3":0.5,"2":1,"21":1,"216":1.5,"2165":1.5},
{"4":0.5,"3":0.5,"32":1,"321":1,"3216":1.5},
{"5":0.5,"4":0.5,"43":0.5,"432":1,"4321":1},
{"6":1,"5":0.5,"54":0.5,"543":0.5,"5432":1},
{"6":1,"1":0.5,"12":1,"123":1,"1234":1},
{"1":0.5,"3":0.5,"34":0.5,"345":0.5,"3456":1},
{"1":0.5,"3":0.5,"4":0.5,"45":0.5,"456":1},
{"1":0.5,"5":0.5,"56":1,"4":0.5,"43":0.5},
{"1":0.5,"6":1,"5":0.5,"54":0.5,"543":0.5},
{"1":0.5,"2":1,"3":0.5,"34":0.5,"345":0.5},
{"1":0.5,"3":0.5,"32":1,"4":0.5,"45":0.5},
{"1":0.5,"4":0.5,"43":0.5,"432":1,"5":0.5},
{"1":0.5,"5":0.5,"54":0.5,"543":0.5,"5432":1}]

In [31]:
#Compare each poly_tree to poly lbm for Ibd

pickle_in = open(r"lbm_data\G62_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}
lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.
hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G62)
G62poly=Polyhedron(ieqs=hyperplanes)

In [32]:
#Properties to distinguish polyhedra

# G62poly.f_vector() 
# G62poly.n_vertices() #6
# G62poly.radius() #0.9128709291752769
# G62poly.representative_point()

#(0.16666666666666666, 0.6666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666)
trans=(-0.16666666666666666, -0.6666666666666666,- 0.16666666666666666, -0.16666666666666666,- 0.16666666666666666, -0.6666666666666666)
transG62poly=G62poly.translation(trans)

pt=transG62poly.representative_point()
pt=[round(i,5) for i in pt]
print(pt)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [33]:
dim_check=[]
pols=[]

print("f-vector G62poly",G62poly.f_vector(),"\n")

for inequ in G62_tree_phi_inequal:
    lbm=[[0,0,0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    tree_poly=Polyhedron(ieqs=hyperp)
    pols.append(tree_poly)
    
#     for i in tree_poly.Hrepresentation():
#         print(i)
    
#     print(dim(G62poly&tree_poly)) #in different hyperplanes. #https://ask.sagemath.org/question/26602/can-i-intersect-the-boundaries-of-two-polyhedra-and-display-it/
    print("Are they iso?",G62poly.is_combinatorially_isomorphic(tree_poly))
    print("f-vector",tree_poly.f_vector()) #Are these polytopes the same?
#     print(tree_poly.n_vertices()) #32
#     print(tree_poly.radius())
    x=tree_poly &transG62poly #Do not contain.
    print("Do they contain each other?",dim(x))
    
    print("rep point",tree_poly.representative_point())
    
#     if x.is_combinatorially_isomorphic(transG62poly):
#         print("contains")
#     else:
#         print("doesnt contain")

    dimp=poly_size(hyperp)
    print("dimension",dimp)
    dim_check.append(dimp)
    print("--------")
print(dim_check) # [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]



f-vector G62poly (1, 6, 15, 20, 15, 6, 1)
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.5, 1.0, 0.0, 0.0, 0.0, -1.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.0, 1.0, 0.5, -2.0, 0.0, 0.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.0, 0.5, 0.5, 0.5, -2.0, 0.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.0, 0.5, 0.0, 0.5, 0.5, -1.5)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (-2.0, 0.5, 0.0, 0.0, 0.5, 1.0)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10, 1)
Do they contain each other? -1
rep point (0.5, 0.5, 0.0, 0.0, -2.0, 1.0)
dimesnsion 5
--------
Are they iso? False
f-vector (1, 32, 80, 80, 40, 10,

In [ ]:
#Comparing phi-tree-polys:

# for p in pols:
#     for q in pols:
#         if p==q:
#             pass
#         else:
# #             print(p==q) #Not equal
# #             print(p.is_combinatorially_isomorphic(q)) #are isomorphic to each other.
#     print("finished this p")

# #Taking intersection of phi-tree-polys
# q=pols[0]
# for p in pols:
#     q=q & p
# print(dim(q)) #5

#Finding a representive point
# r=Polyhedron(ieqs=[(-2,1,1,1,1,1,1),(2,-1,-1,-1,-1,-1,-1)])
# z=q&r
# pt=z.representative_point()
# print(pt)

## G63  and Comparision with polytope by phi lbm inequalities (IBD)

In [ ]:
G63_tree_phi_inequal=[{"1":0.5,"2":0.5,"23":1,"234":1,"2345":1},
{"2":0.5,"3":1,"34":1,"345":1,"3456":1.5},
{"3":1,"4":0.5,"45":0.5,"456":1,"4561":1},
{"4":0.5,"5":0.5,"56":1,"561":1,"5612":1},
{"5":0.5,"6":1,"61":1,"612":1,"6123":1.5},
{"6":1,"1":0.5,"12":0.5,"123":1,"1234":1},
{"4":0.5,"1":0.5,"12":0.5,"123":1,"1236":1.5},
{"4":0.5,"5":0.5,"1":0.5,"12":0.5,"56":1},
{"1":0.5,"12":0.5,"5":0.5,"54":0.5,"6":1},
{"1":0.5,"2":0.5,"23":1,"4":0.5,"45":0.5},
{"2":0.5,"1":0.5,"4":0.5,"5":0.5,"234":0.5},
{"1":0.5,"16":1,"2":0.5,"5":0.5,"54":0.5},
{"2":0.5,"21":0.5,"4":0.5,"45":0.5,"3":1},
{"2":0.5,"21":0.5,"4":0.5,"43":1,"5":0.5},
{"2":0.5,"21":0.5,"216":1,"2163":1.5,"5":0.5}]

In [ ]:
dim_check=[]
for inequ in G63_tree_phi_inequal:
    lbm=[[0,0,0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    dim_check.append(poly_size(hyperp))
print(dim_check)
# [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

## K33 testing dimension of phi-tree-polytope for selected trees

In [41]:
#We choose a selection of trees and determine the dimension of the polytope.

K33_tree_phi_inequal=[{"6":1,"1":1,"12":1.5,"123":2,"1234":2.5},#from necklace
                     {"2":1,"3":1,"25":1.5,"254":2,"2541":2.5,}] #not necklace tree
    
dim_check=[]
for inequ in K33_tree_phi_inequal:
    lbm=[[0,0,0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    
#     for i in hyperp: #Work and cor to paper calc
#         print(i)
#     print("\n")
    
    dim_check.append(poly_size(hyperp))
    
    tree_poly=Polyhedron(ieqs=hyperp)
    
#     for i in tree_poly.Hrepresentation():
#         print(i)
    
#     print("\n rep point",tree_poly.representative_point())
    
    
print(dim_check)


[5, 5]


## $GSym_3$

In [35]:
Gsym3_tree_phi_inequal=[{"1":1,"12":1.5,"123":2}] #from necklace tree
    
dim_check=[]
for inequ in Gsym3_tree_phi_inequal:
    lbm=[[0,0,0,0]]
    hyperp=mod_get_formatted_hyperplanes_txt(lbm,inequ)
    dim_check.append(poly_size(hyperp))
print(dim_check)


[3]


# Get phi that works for Ibd case (phi center of polytope):

We now determine a phi that satisfies the phi inequalities for line bundles applied to the case where the stabiltiy condition is for integral break divisors.

## Get G455 center phi 

We get phi center for all lbm for G455 and complie G455_center_phi as a result.

In [ ]:
pickle_in = open(r"G455_data.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We take G455 and return the phi center of polytope."

phi_G455={"1":1,"2":1.5,"3":1,"4":1.5,"12":1.5,"23":1.5,"34":1.5,"41":1.5}

all_lbm=[i[1] for i in data]

for lbm in all_lbm:
    
    data=get_formatted_hyperplanes_txt(lbm,phi_G455)

    poly=Polyhedron(ieqs=data)
    phi=get_a_phi(poly) # this is not the average
    # 
    print(lbm)
    mod_where_specific_lbm_fail(phi_G455,lbm,phi)
    
    print("\n ----")

## G62

In [ ]:
pickle_in = open(r"lbm_data\G62_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We check if another phi works"

phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}

lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G62)

poly=Polyhedron(ieqs=hyperplanes)
phi=get_a_phi(poly)

# print(mod_where_specific_lbm_fail(phi_G62,lbm,phi))

"""
G62
The phi (center of polytope): (1/6,2/3,1/6,1/6,1/6,2/3)  does not fail the phi-inequalities
phi average is: [0.35714286 0.5        0.21428571 0.21428571 0.21428571 0.5       ]
"""

## G63

In [ ]:
pickle_in = open(r"lbm_data\G63_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We check if another phi works"

lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.

phi_G63={"1":1,"2":1,"3":1.5,"4":1,"5":1,"6":1.5,"12":1,"23":1.5,"34":1.5,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1.5,"456":1.5,"561":1.5,"612":1.5}

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G63)

poly=Polyhedron(ieqs=hyperplanes)

phi=get_a_phi(poly)

# phi_av = [0.26666667 0.26666667 0.46666667 0.26666667 0.26666667 0.46666667]

mod_where_specific_lbm_fail(phi_G63,lbm,phi)

"""
G63
The phi (center of polytope): (1/6,1/6,2/3,1/6,1/6,2/3) does not fail the phi-inequalities
(0.16666666666666666, 0.16666666666666666, 0.6666666666666666, 0.16666666666666666, 0.16666666666666666, 0.6666666666666666) 
phi average is: [0.26666667 0.26666667 0.46666667 0.26666667 0.26666667 0.46666667]
"""

## G72

In [ ]:
pickle_in = open(r"lbm_data\G72_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We check if another phi works"

lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.

phi_G72={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1,"7":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1,"67":1.5,"71":1.5,"123":1.5,"234":1.5,"345":1,"456":1,"567":1.5,"671":1.5,"712":1}

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G72)

poly=Polyhedron(ieqs=hyperplanes)

phi=get_a_phi(poly)

mod_where_specific_lbm_fail(phi_G72,lbm,phi)

"""
G72
The phi (center of polytope): (1/7,9/14,1/7,1/7,1/7,1/7,9/14) does not fail the phi-inequalities
(0.14285714285714285, 0.6428571428571428, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.6428571428571428) 
phi average is: [0.35294118 0.47058824 0.17647059 0.17647059 0.17647059 0.17647059 0.47058824]
"""

## G73

In [ ]:
pickle_in = open(r"lbm_data\G73_phi_failcases_lbm.txt", "rb")
data = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
"We check if another phi works"

lbm=data[0] # print(data[0]) #the first element is that of Integral break divisors.

phi_G73={"1":1,"2":1,"3":1.5,"4":1,"5":1,"6":1,"7":1.5,"12":1,"23":1.5,"34":1.5,"45":1,"56":1,"67":1.5,"71":1.5,"123":1.5,"234":1.5,"345":1.5,"456":1,"567":1.5,"671":1.5,"712":1.5}

hyperplanes=get_formatted_hyperplanes_txt(lbm,phi_G73)

poly=Polyhedron(ieqs=hyperplanes)

phi=get_a_phi(poly)

mod_where_specific_lbm_fail(phi_G73,lbm,phi)

"""
G73
The phi (center of polytope): (1/7,1/7,9/14,1/7,1/7,1/7,9/14)  does not fail the phi-inequalities
(0.14285714285714285, 0.14285714285714285, 0.6428571428571428, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.6428571428571428) 
phi average is: [0.26315789 0.26315789 0.42105263 0.21052632 0.21052632 0.21052632 0.42105263]
"""